# Knowledge Representation

## Einführung
WIP

## Arbeitsaufteilung
WIP


### 1. Installation und Importieren der benötigten Libraries

In [ ]:
!pip install -r requirements.txt

### 2. WIP

### 3. WIP

### 4. WIP

### 5. WIP

# Ressourcen & Source-Docs

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

# Challenges bei der Implementierung


#### Cesar Laura, Scraping


#### Dilly Julian, Datenbereinigung


#### Ecker Annina, Implementierung von Selenium:
